<a href="https://colab.research.google.com/github/NHMSudara/CNN-model-for-Brachial-Plexus-Nerve-Trunks-Recognition-in-Ultrasound-Images/blob/master/Inc%2BResUNet_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Unzip dataset


In [ ]:
!unzip /content/drive/MyDrive/train_1.zip

Streaming output truncated to the last 5000 lines.
  inflating: train/33_114.tif        
  inflating: train/33_114_mask.tif   
  inflating: train/33_115.tif        
  inflating: train/33_115_mask.tif   
  inflating: train/33_116.tif        
  inflating: train/33_116_mask.tif   
  inflating: train/33_117.tif        
  inflating: train/33_117_mask.tif   
  inflating: train/33_118.tif        
  inflating: train/33_118_mask.tif   
  inflating: train/33_119.tif        
  inflating: train/33_119_mask.tif   
  inflating: train/33_12.tif         
  inflating: train/33_12_mask.tif    
  inflating: train/33_120.tif        
  inflating: train/33_120_mask.tif   
  inflating: train/33_13.tif         
  inflating: train/33_13_mask.tif    
  inflating: train/33_14.tif         
  inflating: train/33_14_mask.tif    
  inflating: train/33_15.tif         
  inflating: train/33_15_mask.tif    
  inflating: train/33_16.tif         
  inflating: train/33_16_mask.tif    
  inflating: train/33_17.tif         

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/fyp_my/new_data_nr.zip

Streaming output truncated to the last 5000 lines.
  inflating: new_data_nr/train/mask/4663_0.jpg  
  inflating: new_data_nr/train/mask/4663_1.jpg  
  inflating: new_data_nr/train/mask/4663_2.jpg  
  inflating: new_data_nr/train/mask/4663_3.jpg  
  inflating: new_data_nr/train/mask/4663_4.jpg  
  inflating: new_data_nr/train/mask/4664_0.jpg  
  inflating: new_data_nr/train/mask/4664_1.jpg  
  inflating: new_data_nr/train/mask/4664_2.jpg  
  inflating: new_data_nr/train/mask/4664_3.jpg  
  inflating: new_data_nr/train/mask/4664_4.jpg  
  inflating: new_data_nr/train/mask/4665_0.jpg  
  inflating: new_data_nr/train/mask/4665_1.jpg  
  inflating: new_data_nr/train/mask/4665_2.jpg  
  inflating: new_data_nr/train/mask/4665_3.jpg  
  inflating: new_data_nr/train/mask/4665_4.jpg  
  inflating: new_data_nr/train/mask/4666_0.jpg  
  inflating: new_data_nr/train/mask/4666_1.jpg  
  inflating: new_data_nr/train/mask/4666_2.jpg  
  inflating: new_data_nr/train/mask/4666_3.jpg  
  inflating: new_d

# Data pre-process funtions & loss funtions

In [ ]:
import tensorflow as tf
import os
import numpy as np
from PIL import Image
import cv2
from numpy import asarray
from sklearn.model_selection import train_test_split

def load_data(path):
    x = sorted(glob(os.path.join(path, "image", "*.jpg")))
    y = sorted(glob(os.path.join(path, "mask", "*.jpg")))
    return x, y


def PreprocessData(img,mask,target_shape_img, target_shape_mask,path1):

    print(img)
    print(len(img))
    # Pull the relevant dimensions for image and mask
    m = len(img)                     # number of images
    i_h,i_w,i_c = target_shape_img   # pull height, width, and channels of image
    m_h,m_w,m_c = target_shape_mask  # pull height, width, and channels of mask

    # Define X and Y as number of images along with shape of one image
    X = np.zeros((m,i_h,i_w,i_c), dtype=np.float32)
    y = np.zeros((m,m_h,m_w,m_c), dtype=np.int32)
    index=0
    # Resize images and masks
    for file in img:
        # convert image into an array of desired shape (3 channels)


        sr = file.split('/')[5]


        dotsplit = sr.split('.')[0]
        sec=dotsplit.split('_')[-1]
        fir=dotsplit.split('_')[0]
        s=int(sec)
        f=int(fir)

        index= ((s*(f-1))+s)
        path = os.path.join(path1, file)
        single_img = Image.open(path).convert('RGB')
        single_img = single_img.resize((i_h,i_w))
        single_img = np.reshape(single_img,(i_h,i_w,i_c))
        single_img = single_img/255
        X[index] = single_img

        # convert mask into an array of desired shape (1 channel)

        single_mask_ind = mask[index]
        path = os.path.join(path1, single_mask_ind)
        single_mask = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        single_mask = cv2.resize(single_mask, dsize=(m_h, m_w), interpolation=cv2.INTER_NEAREST)
        single_mask = asarray(single_mask)
        single_mask = single_mask[..., tf.newaxis]
        single_mask = np.reshape(single_mask,(m_h,m_w,m_c))
        single_mask = (single_mask > 0)

        y[index] = single_mask
        index=index+1
    return X, y




smooth = 1
def dice_coef(y_true, y_pred):
    y_true_float32 = tf.cast(y_true, tf.float32)
    y_pred_float32 = tf.cast(y_pred, tf.float32)
    y_true = tf.keras.layers.Flatten()( y_true_float32)
    y_pred = tf.keras.layers.Flatten()(y_pred_float32)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)


old

In [ ]:
import tensorflow as tf
import os
import numpy as np
from PIL import Image
import cv2
from numpy import asarray
from sklearn.model_selection import train_test_split



def check_mask(imagename,cat_num):
        dotsplit = imagename.split('.')[0]
        mask_split=dotsplit.split('_')[-1]
        cat_number=dotsplit.split('_')[0]
        if (mask_split=='mask' and cat_number==cat_num):
            return 1
        elif (not(mask_split=='mask') and cat_number==cat_num):
            return 2


def LoadData (path1,cat_num):                                 # divide images in to original image and mask(label)
    # Read the images folder like a list
    image_dataset = os.listdir(path1)

    # Make a list for images and masks filenames
    orig_img = []
    mask_img = []
    image_dataset.sort()
    for file in image_dataset:
        if ((check_mask(file,cat_num)) == 1):
            mask_img.append(file)
        elif (check_mask(file,cat_num)==2):
            orig_img.append(file.replace("_mask.tif",".tif"))

    return orig_img, mask_img





def order_image_mask_list(path1,cat_len):                   # give order list of image names and mask
    image_dataset = os.listdir(path1)
    list=np.zeros(47,dtype=int)
    nlist=np.zeros(int(len(image_dataset)*0.5),dtype=object)
    mlist=np.zeros(int(len(image_dataset)*0.5),dtype=object)
    print(len(nlist))


    # count the each category of image containe number of images
    for i in image_dataset:
        dotsplit = i.split('.')[0]
        if (not(dotsplit.split('_')[-1]=='mask')):
            v=int(dotsplit.split('_')[0])                   # seperate number under category (second number)
            list[v-1]= list[v-1]+1


    val=0
    img_zero=0

    mval=0
    mask_zero=0

    for u in range(cat_len):                                     # 47 is number of categories of images (First number of image name '13_53.tif'==>13)
        img, mask = LoadData (path1,str(u+1))               # get divide images in to original image and mask(label)
        for i in img:
            dotsplit = i.split('.')[0]
            ind=(dotsplit.split('_')[1])
            index=int(ind)
            if(not(u==0) and img_zero ==0 ):                      # identify first image position and it give to val variable
                val=list[u-1]+val
                img_zero=1
            nlist[val+index-1]=i                                  # normal image name list assending order
        img_zero=0

        for k in mask:
            dotsplit = k.split('.')[0]
            index=int(dotsplit.split('_')[1])
            if(not(u==0) and mask_zero ==0 ):
                mval=list[u-1]+mval
                mask_zero=1
            mlist[mval+index-1]= k                                  # mask name list assending order
        mask_zero=0


    img=nlist
    mask=mlist
    print(type(img))
    return nlist,mlist


def PreprocessData(img,mask,target_shape_img, target_shape_mask,path1):

    print(img)
    print(len(img))
    # Pull the relevant dimensions for image and mask
    m = len(img)                     # number of images
    i_h,i_w,i_c = target_shape_img   # pull height, width, and channels of image
    m_h,m_w,m_c = target_shape_mask  # pull height, width, and channels of mask

    # Define X and Y as number of images along with shape of one image
    X = np.zeros((m,i_h,i_w,i_c), dtype=np.float32)
    y = np.zeros((m,m_h,m_w,m_c), dtype=np.int32)
    index=0
    # Resize images and masks
    for file in img:
        # convert image into an array of desired shape (3 channels)

        path = os.path.join(path1, file)
        single_img = Image.open(path).convert('RGB')
        single_img = single_img.resize((i_h,i_w))
        single_img = np.reshape(single_img,(i_h,i_w,i_c))
        single_img = single_img/255
        X[index] = single_img

        # convert mask into an array of desired shape (1 channel)

        single_mask_ind = mask[index]
        path = os.path.join(path1, single_mask_ind)
        single_mask = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        single_mask = cv2.resize(single_mask, dsize=(m_h, m_w), interpolation=cv2.INTER_NEAREST)
        single_mask = asarray(single_mask)
        single_mask = single_mask[..., tf.newaxis]
        single_mask = np.reshape(single_mask,(m_h,m_w,m_c))
        single_mask = (single_mask > 0)

        y[index] = single_mask
        index=index+1
    return X, y

#pre process Y_train and Y_test for classification model, It only include '1' and '0'
def classification_pre(Y):
    i=0
    classi_test=np.zeros(len(Y),dtype=int)
    for mask in Y:
        if sum(Y[i].flatten()) > 0 :
            classi_test[i]=1
        else:
            classi_test[i]=0
        i+=1
    return classi_test



smooth = 1
def dice_coef(y_true, y_pred):
    y_true_float32 = tf.cast(y_true, tf.float32)
    y_pred_float32 = tf.cast(y_pred, tf.float32)
    y_true = tf.keras.layers.Flatten()( y_true_float32)
    y_pred = tf.keras.layers.Flatten()(y_pred_float32)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)


# Data load

In [ ]:
from glob import glob
dataset_path = "/content/new_data_nr"
train_path = os.path.join(dataset_path, "train")
valid_path = os.path.join(dataset_path, "test")
# Define the desired shape
target_shape_img = [96, 96, 3]
target_shape_mask = [96, 96, 1]

train_x, train_y = load_data(train_path)
valid_x, valid_y = load_data(valid_path)

X_train, Y_train = PreprocessData(train_x,train_y,target_shape_img, target_shape_mask, train_path)
X_test, Y_test = PreprocessData(valid_x,valid_y,target_shape_img, target_shape_mask, valid_path)

['/content/new_data_nr/train/image/1000_0.jpg', '/content/new_data_nr/train/image/1000_1.jpg', '/content/new_data_nr/train/image/1000_2.jpg', '/content/new_data_nr/train/image/1000_3.jpg', '/content/new_data_nr/train/image/1000_4.jpg', '/content/new_data_nr/train/image/1001_0.jpg', '/content/new_data_nr/train/image/1001_1.jpg', '/content/new_data_nr/train/image/1001_2.jpg', '/content/new_data_nr/train/image/1001_3.jpg', '/content/new_data_nr/train/image/1001_4.jpg', '/content/new_data_nr/train/image/1002_0.jpg', '/content/new_data_nr/train/image/1002_1.jpg', '/content/new_data_nr/train/image/1002_2.jpg', '/content/new_data_nr/train/image/1002_3.jpg', '/content/new_data_nr/train/image/1002_4.jpg', '/content/new_data_nr/train/image/1003_0.jpg', '/content/new_data_nr/train/image/1003_1.jpg', '/content/new_data_nr/train/image/1003_2.jpg', '/content/new_data_nr/train/image/1003_3.jpg', '/content/new_data_nr/train/image/1003_4.jpg', '/content/new_data_nr/train/image/1004_0.jpg', '/content/ne

old

In [ ]:
path1 = '/content/train' #define path
# Define the desired shape
target_shape_img = [96, 96, 3]
target_shape_mask = [96, 96, 1]

img,mask  = order_image_mask_list(path1,47)

X, Y = PreprocessData(img,mask,target_shape_img, target_shape_mask, path1)
X_train,X_test, Y_train,Y_test= train_test_split(X,Y,test_size=0.2,random_state=10)

5635
<class 'numpy.ndarray'>
['1_1.tif' '1_2.tif' '1_3.tif' ... '47_118.tif' '47_119.tif' '47_120.tif']
5635


## Inc+ResUNet Model

In [ ]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
import tensorflow as tf
import joblib
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
import os
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout,Activation, Concatenate,Add
import tensorflow as tf
import joblib
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
import os

def block(prevlayer, a, b, pooling):
    conva = Conv2D(a, (7, 7), activation='relu', padding='same')(prevlayer)
    conva = BatchNormalization()(conva)
    conva = Conv2D(b, (7, 7), activation='relu', padding='same')(conva)
    conva = BatchNormalization()(conva)
    if True == pooling:
        conva = MaxPooling2D(pool_size=(2, 2))(conva)


    convb = Conv2D(a, (5, 5), activation='relu', padding='same')(prevlayer)
    convb = BatchNormalization()(convb)
    convb = Conv2D(b, (5, 5), activation='relu', padding='same')(convb)
    convb = BatchNormalization()(convb)
    if True == pooling:
        convb = MaxPooling2D(pool_size=(2, 2))(convb)

    convc = Conv2D(b, (1, 1), activation='relu', padding='same')(prevlayer)
    convc = BatchNormalization()(convc)
    if True == pooling:
        convc = MaxPooling2D(pool_size=(2, 2))(convc)

    convd = Conv2D(a, (3, 3), activation='relu', padding='same')(prevlayer)
    convd = BatchNormalization()(convd)
    convd = Conv2D(b, (1, 1), activation='relu', padding='same')(convd)
    convd = BatchNormalization()(convd)
    if True == pooling:
        convd = MaxPooling2D(pool_size=(2, 2))(convd)
    convs = Conv2D((b*4), (1, 1), activation='relu', padding='same')(prevlayer)
    if True == pooling:
        convs = MaxPooling2D(pool_size=(2, 2))(convs)
    up = concatenate([conva, convb, convc, convd])
    return up,convs


    return conv


img_rows = 96
img_cols = 96
depth = 3

def res_block(x1,x2):
  return Add()([x1,x2])


def create_unet_model():
    inputs = Input((img_rows, img_cols, depth))

    xx1,s = block(inputs, 16, 16, False)
    x1=res_block(xx1,s)


    xx2,s1 = block(xx1, 32, 32, True)
    x2=res_block(xx2,s1)


    xx3,s2 = block(xx2, 64, 64,True)
    x3=res_block(xx3,s2)


    xx4,s3 = block(xx3, 128, 128, True)
    x4=res_block(xx4,s3)


    up1 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(x4), x3], axis=3)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up1)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

    up2 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), x2], axis=3)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up2)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

    up3 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), x1], axis=3)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up3)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv8)
    model = Model(inputs=[inputs], outputs=[conv10])


    return model

unet_model=create_unet_model()
print("Model created...")
unet_model.summary()

unet_model.compile(optimizer=tf.keras.optimizers.Adam(
    learning_rate=0.00001,), loss=dice_loss, metrics=[dice_coef])
model_path = os.path.join("files", "unet_model.h5")
csv_path = os.path.join("files", "unet_log.csv")
callbacks = [
    ModelCheckpoint(model_path, verbose=1, save_best_only=True),
    CSVLogger(csv_path),
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=False),
]

history=unet_model.fit(X_train,Y_train, batch_size=32, epochs=10, verbose=1, shuffle=False,
              validation_data= [X_test,Y_test],callbacks=callbacks
             )

filename = 'unet_model.joblib'
joblib.dump(unet_model, filename)

Model created...
Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 96, 96, 3)]  0           []                               
                                                                                                  
 conv2d_78 (Conv2D)             (None, 96, 96, 16)   2368        ['input_3[0][0]']                
                                                                                                  
 conv2d_80 (Conv2D)             (None, 96, 96, 16)   4816        ['input_3[0][0]']                
                                                                                                  
 conv2d_83 (Conv2D)             (None, 96, 96, 16)   448         ['input_3[0][0]']                
                                                                           

['unet_model.joblib']

In [ ]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
import tensorflow as tf
import joblib
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
import os
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout,Activation, Concatenate,Add
import tensorflow as tf
import joblib
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
import os

def block(prevlayer, a, b, pooling):
    conva = Conv2D(a, (5, 5), activation='relu', padding='same')(prevlayer)
    conva = BatchNormalization()(conva)
    conva = Conv2D(b, (5, 5), activation='relu', padding='same')(conva)
    conva = BatchNormalization()(conva)
    if True == pooling:
        conva = MaxPooling2D(pool_size=(2, 2))(conva)


    convb = Conv2D(a, (3, 3), activation='relu', padding='same')(prevlayer)
    convb = BatchNormalization()(convb)
    convb = Conv2D(b, (3, 3), activation='relu', padding='same')(convb)
    convb = BatchNormalization()(convb)
    if True == pooling:
        convb = MaxPooling2D(pool_size=(2, 2))(convb)

    convc = Conv2D(b, (1, 1), activation='relu', padding='same')(prevlayer)
    convc = BatchNormalization()(convc)
    if True == pooling:
        convc = MaxPooling2D(pool_size=(2, 2))(convc)

    convd = Conv2D(a, (2, 2), activation='relu', padding='same')(prevlayer)
    convd = BatchNormalization()(convd)
    convd = Conv2D(b, (1, 1), activation='relu', padding='same')(convd)
    convd = BatchNormalization()(convd)
    if True == pooling:
        convd = MaxPooling2D(pool_size=(2, 2))(convd)
    convs = Conv2D((b*4), (1, 1), activation='relu', padding='same')(prevlayer)
    if True == pooling:
        convs = MaxPooling2D(pool_size=(2, 2))(convs)
    up = concatenate([conva, convb, convc, convd])
    return up,convs


    return conv


img_rows = 96
img_cols = 96
depth = 3

def res_block(x1,x2):
  return Add()([x1,x2])


def create_inc_resunet_model():
    inputs = Input((img_rows, img_cols, depth))

    xx1,s = block(inputs, 16, 16, False)
    x1=res_block(xx1,s)


    xx2,s1 = block(xx1, 32, 32, True)
    x2=res_block(xx2,s1)


    xx3,s2 = block(xx2, 64, 64,True)
    x3=res_block(xx3,s2)


    xx4,s3 = block(xx3, 128, 128, True)
    x4=res_block(xx4,s3)


    up1 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(x4), x3], axis=3)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up1)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

    up2 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), x2], axis=3)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up2)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

    up3 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), x1], axis=3)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up3)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv8)
    model = Model(inputs=[inputs], outputs=[conv10])


    return model

inc_resunet_model=create_inc_resunet_model()
print("Model created...")
inc_resunet_model.summary()




inc_resunet_model.compile(optimizer=tf.keras.optimizers.Adam(
    learning_rate=0.00001,), loss=dice_loss, metrics=[dice_coef])
model_path = os.path.join("files", "in_resmodel.h5")
csv_path = os.path.join("files", "in_reslog.csv")
callbacks = [
    ModelCheckpoint(model_path, verbose=1, save_best_only=True),
    CSVLogger(csv_path),
    EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False),
]


file_name = 'inc_resUNet'
tensorboard = TensorBoard(log_dir="logs_{}".format(file_name))

history=inc_resunet_model.fit(X_train, Y_train, batch_size=32, epochs=20, verbose=1, shuffle=False,
              validation_data= [X_test, Y_test],callbacks=[callbacks,tensorboard])


Model created...
Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 96, 96, 3)]  0           []                               
                                                                                                  
 conv2d_78 (Conv2D)             (None, 96, 96, 16)   1216        ['input_3[0][0]']                
                                                                                                  
 conv2d_80 (Conv2D)             (None, 96, 96, 16)   448         ['input_3[0][0]']                
                                                                                                  
 conv2d_83 (Conv2D)             (None, 96, 96, 16)   208         ['input_3[0][0]']                
                                                                           

In [ ]:
%cd /content/

/content


In [ ]:
!zip -r logs_inc_resUNetwgs.zip logs_inc_resUNet

  adding: logs_inc_resUNet/ (stored 0%)
  adding: logs_inc_resUNet/validation/ (stored 0%)
  adding: logs_inc_resUNet/validation/events.out.tfevents.1694020910.fd7c7d40518e.413.4.v2 (deflated 75%)
  adding: logs_inc_resUNet/validation/events.out.tfevents.1694019323.fd7c7d40518e.413.1.v2 (deflated 75%)
  adding: logs_inc_resUNet/train/ (stored 0%)
  adding: logs_inc_resUNet/train/events.out.tfevents.1694020655.fd7c7d40518e.413.2.v2 (deflated 95%)
  adding: logs_inc_resUNet/train/events.out.tfevents.1694020840.fd7c7d40518e.413.3.v2 (deflated 94%)
  adding: logs_inc_resUNet/train/events.out.tfevents.1694019225.fd7c7d40518e.413.0.v2 (deflated 94%)


In [ ]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
import tensorflow as tf
import joblib
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
import os
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout,Activation, Concatenate,Add
import tensorflow as tf
import joblib
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
import os

def block(prevlayer, a, b, pooling):
    conva = Conv2D(a, (5, 5), activation='relu', padding='same')(prevlayer)
    conva = BatchNormalization()(conva)
    conva = Conv2D(b, (5, 5), activation='relu', padding='same')(conva)
    conva = BatchNormalization()(conva)
    if True == pooling:
        conva = MaxPooling2D(pool_size=(2, 2))(conva)


    convb = Conv2D(a, (3, 3), activation='relu', padding='same')(prevlayer)
    convb = BatchNormalization()(convb)
    convb = Conv2D(b, (3, 3), activation='relu', padding='same')(convb)
    convb = BatchNormalization()(convb)
    if True == pooling:
        convb = MaxPooling2D(pool_size=(2, 2))(convb)

    convc = Conv2D(b, (1, 1), activation='relu', padding='same')(prevlayer)
    convc = BatchNormalization()(convc)
    if True == pooling:
        convc = MaxPooling2D(pool_size=(2, 2))(convc)

    convd = Conv2D(a, (2, 2), activation='relu', padding='same')(prevlayer)
    convd = BatchNormalization()(convd)
    convd = Conv2D(b, (1, 1), activation='relu', padding='same')(convd)
    convd = BatchNormalization()(convd)
    if True == pooling:
        convd = MaxPooling2D(pool_size=(2, 2))(convd)
    convs = Conv2D((b*4), (1, 1), activation='relu', padding='same')(prevlayer)
    if True == pooling:
        convs = MaxPooling2D(pool_size=(2, 2))(convs)
    up = concatenate([conva, convb, convc, convd])
    return up,convs


    return conv


img_rows = 96
img_cols = 96
depth = 3

def res_block(x1,x2):
  return Add()([x1,x2])


def create_inc_resunet_model():
    inputs = Input((img_rows, img_cols, depth))

    xx1,s = block(inputs, 16, 16, False)
    x1=res_block(xx1,s)


    xx2,s1 = block(xx1, 32, 32, True)
    x2=res_block(xx2,s1)


    xx3,s2 = block(xx2, 64, 64,True)
    x3=res_block(xx3,s2)


    xx4,s3 = block(xx3, 128, 128, True)
    x4=res_block(xx4,s3)


    up1 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(x4), x3], axis=3)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up1)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

    up2 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), x2], axis=3)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up2)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

    up3 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), x1], axis=3)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up3)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv8)
    model = Model(inputs=[inputs], outputs=[conv10])


    return model

inc_resunet_model=create_inc_resunet_model()
print("Model created...")
#inc_resunet_model.summary()
from keras.utils.vis_utils import plot_model
graph=plot_model(inc_resunet_model,'incpres.png',show_shapes=True)

Model created...
